In [2]:
import numpy as np

def rossler(xyz, a=0.2, b=0.2, c=5.7):
    x, y, z = xyz
    return np.array([-y - z, x + a*y, b + z*(x - c)], dtype=np.float64)

def J_dm_x(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [1.0, 0.0, 0.0],
        [0.0, -1.0, -1.0],
        [-(1.0 + z), -a, -(x - c)]
    ], dtype=np.float64)

def J_dm_y(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [0.0, 1.0, 0.0],
        [1.0, a,   0.0],
        [a,   (a*a - 1.0), -1.0]
    ], dtype=np.float64)

def J_dm_z(x, y, z, a=0.2, b=0.2, c=5.7):
    u = x - c
    return np.array([
        [0.0, 0.0, 1.0],
        [z,   0.0, u],
        [b + 2.0*z*u,  -z,   (u*u - y - 2.0*z)]
    ], dtype=np.float64)

def J_dm_xy(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [1.0, 1.0, 0.0],
        [1.0, (a-1.0), -1.0],
        [(a-1.0) - z, (a*a - a + 1.0), (c-1.0) - x]
    ], dtype=np.float64)

def J_dm_yz(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [0.0, 1.0, 1.0],
        [1.0 + z, a, (-c + x)],
        [(a+b) - (2.0*c + 1.0)*z + 2.0*x*z, (a*a - 1.0), (c*c - 1.0) - (2.0*c + 1.0)*x - 2.0*z + x*x]
    ], dtype=np.float64)

def J_dm_zx(x, y, z, a=0.2, b=0.2, c=5.7):
    u = x - c
    return np.array([
        [1.0, 0.0, 1.0],
        [z,   -1.0, (u - 1.0)],
        [(b-1.0) + (1.0 - 2.0*c)*z, (-a - z), c*(c+1.0) + (1.0 - 2.0*c)*x - y - 2.0*z]
    ], dtype=np.float64)

def J_x_y_x(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, -1.0, -1.0]
    ], dtype=np.float64)

def J_x_y_y(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [1.0, a,   0.0]
    ], dtype=np.float64)

def J_y_z_y(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [1.0, a,   0.0]
    ], dtype=np.float64)

def J_y_z_z(x, y, z, a=0.2, b=0.2, c=5.7):
    u = x - c
    return np.array([
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [z,   0.0, u]
    ], dtype=np.float64)

def J_x_z_z(x, y, z, a=0.2, b=0.2, c=5.7):
    u = x - c
    return np.array([
        [1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0],
        [z,   0.0, u]
    ], dtype=np.float64)

def J_x_z_x(x, y, z, a=0.2, b=0.2, c=5.7):
    return np.array([
        [1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, -1.0, -1.0]
    ], dtype=np.float64)

def letellier_observability(J_func, traj, burn_in=2000, a=0.2, b=0.2, c=5.7):
    T = traj.shape[0]
    deltas = np.zeros(T, dtype=np.float64)
    for i in range(burn_in, T):
        x, y, z = traj[i]
        Q = J_func(x, y, z, a=a, b=b, c=c)
        eigs = np.linalg.eigvalsh(Q.T @ Q)
        deltas[i] = eigs[0] / eigs[-1]
    return float(np.mean(deltas[burn_in:])), deltas

dt = 0.01
num_steps = int(2e4)
traj = np.zeros((num_steps + 1, 3), dtype=np.float64)
traj[0] = np.array([1.0, 1.0, 0.0], dtype=np.float64)

for i in range(num_steps):
    traj[i + 1] = traj[i] + rossler(traj[i]) * dt

JACOBIANS = {
    "dm_x":   J_dm_x,
    "dm_y":   J_dm_y,
    "dm_z":   J_dm_z,
    "dm_xy":  J_dm_xy,
    "dm_yz":  J_dm_yz,
    "dm_zx":  J_dm_zx,
    "x_y_x":  J_x_y_x,
    "x_y_y":  J_x_y_y,
    "y_z_y":  J_y_z_y,
    "y_z_z":  J_y_z_z,
    "x_z_z":  J_x_z_z,
    "x_z_x":  J_x_z_x,
}

results = []
for name, Jf in JACOBIANS.items():
    D, _ = letellier_observability(Jf, traj)
    results.append((name, D))

results_sorted = sorted(results, key=lambda t: t[1], reverse=True)

for name, D in results_sorted:
    print(f"{name:6s} : D = {D:.2f}")

y_z_y  : D = 0.67
x_y_x  : D = 0.15
x_z_x  : D = 0.15
dm_y   : D = 0.14
dm_xy  : D = 0.05
dm_x   : D = 0.03
y_z_z  : D = 0.01
dm_yz  : D = 0.01
dm_zx  : D = 0.00
dm_z   : D = 0.00
x_y_y  : D = 0.00
x_z_z  : D = -0.00
